## Логистическая регрессия

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score

In [2]:
def write_answer(filename, answer):
    
    with open(filename, 'w') as fout:
        
        fout.write(str(answer))
        fout.close()

### Введение

Логистическая регрессия — один из видов линейных классификаторов. Одной из ее особенностей является возможность оценивания вероятностей классов, тогда как большинство линейных классификаторов могут выдавать только номера классов. 

Логистическая регрессия использует достаточно сложный функционал качества, который не допускает записи решения в явном виде (в отличие от, например, линейной регрессии). Тем не менее, логистическую регрессию можно настраивать с помощью градиентного спуска.

Мы будем работать с выборкой, содержащей два признака. Будем считать, что ответы лежат в множестве $\{-1, 1\}$. Для настройки логистической регрессии мы будем решать следующую задачу:

$$\frac{1}{l}\sum_{i=1}^l log(1+exp(-y_i(w_1 x_{i1} + w_2 x_{i2}))) + \frac{1}{2}C||w||^2 \rightarrow \min_{w_1, w_2}$$


Здесь $x_{i1}$ и $x_{i2}$ — значение первого и второго признаков соответственно на объекте $x_i$. В этом задании мы будем рассматривать алгоритмы без свободного члена, чтобы упростить работу.

Градиентный шаг для весов будет заключаться в одновременном обновлении весов $w_1$ и $w_2$ по следующим формулам (проверьте сами, что здесь действительно выписана производная нашего функционала):

$$w_1 := w_1 + k\frac{1}{l}\sum_{i=1}^l y_i x_{i1} (1 - \frac{1}{1+exp(-y_i(w_1 x_{i1} + w_2 x_{i2}))}) -kC w_1$$
$$w_2 := w_2 + k\frac{1}{l}\sum_{i=1}^l y_i x_{i2} (1 - \frac{1}{1+exp(-y_i(w_1 x_{i1} + w_2 x_{i2}))}) -kC w_2$$


Здесь k — размер шага. 

Линейные методы могут переобучаться и давать плохое качество из-за различных проблем в данных: мультиколлинеарности, зашумленности и т.д. Чтобы избежать этого, следует использовать регуляризацию — она позволяет понизить сложность модели и не допустить переобучения. Сила регуляризации определяется коэффициентом C в формулах, указанных выше.


### Реализация в Scikit-Learn

В этом задании мы предлагаем вам самостоятельно реализовать градиентный спуск. 

В качестве метрики качества будем использовать <b>AUC-ROC (Area Under ROC-Curve)</b>. Она предназначена для алгоритмов бинарной классификации, выдающих оценку принадлежности объекта к одному из классов. По сути, значение этой метрики является агрегацией показателей качества всех алгоритмов, которые можно получить, выбирая какой-либо порог для оценки принадлежности.

В Scikit-Learn метрика AUC реализована функцией <code>sklearn.metrics.roc_auc_score</code>. В качестве первого аргумента ей передается вектор истинных ответов, в качестве второго — вектор с оценками принадлежности объектов к первому классу.

#### 1. Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [3]:
train = pd.read_csv('data-logistic.csv', header=None)
X, y = train.iloc[:, 1:], train.iloc[:, 0]

#### 2. Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).
Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.

In [4]:
def sigmoid(M):
    return 1.0 / (1.0 + np.exp(-M))

def sigma(i, X, y, w1, w2):
    return 1.0 / (1.0 + np.exp(-y[i] * (w1*X[1][i] + w2*X[2][i])))

def grad_sum_w1(X, y, w1, w2):
    return sum((y[i] * X[1][i] * (1.0 - sigma(i, X, y, w1, w2)) for i in np.arange(0, len(y))))

def grad_sum_w2(X, y, w1, w2):
    return sum((y[i] * X[2][i] * (1.0 - sigma(i, X, y, w1, w2)) for i in np.arange(0, len(y))))

def log_regression(X, y, k=0.1, w1=0.0, w2=0.0, C=0, epsilon=1e-5, max_iter=10000):

    for i in range(max_iter):
        
        w1_prev, w2_prev = w1, w2
        
        w1 =  w1 + k / len(y) * grad_sum_w1(X, y, w1, w2) - k * C * w1
        w2 =  w2 + k / len(y) * grad_sum_w2(X, y, w1, w2) - k * C * w2
        
        if np.sqrt((w1_prev - w1) ** 2 + (w2_prev - w2) ** 2) <= epsilon:
            print(f'Convergence iteration with C={C}: {i}')
            break
    
    w = np.array([w1, w2])

    predictions = sigmoid(np.dot(X, w))
    return predictions

#### 3. Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)). 

In [5]:
predictions = log_regression(X=X, y=y)
reg_predictions = log_regression(X=X, y=y, C=10)

roc_auc = roc_auc_score(y, predictions)
roc_auc_reg = roc_auc_score(y, reg_predictions)

print(f'ROC-AUC Score without regularization: {roc_auc}')
print(f'ROC-AUC Score with L2 regularization: {roc_auc_reg}')

write_answer('submission_logreg_1.txt', f'{round(roc_auc, 3)} {round(roc_auc_reg, 3)}')

Convergence iteration with C=0: 235
Convergence iteration with C=10: 5
ROC-AUC Score without regularization: 0.9268571428571428
ROC-AUC Score with L2 regularization: 0.9362857142857142


Качество классификатора при использовании регуляризации выше, чем без нее.

#### 4. Попробуйте поменять длину шага. Будет ли сходиться алгоритм, если делать более длинные шаги? Как меняется число итераций при уменьшении длины шага?

Изменим длину шага(learning rate) k

In [6]:
for k in [1e-2, 1e-3, 1e-4]:
    
    reg_predictions = log_regression(X=X, y=y, C=10, k=k)

    roc_auc = roc_auc_score(y, reg_predictions)

    print(f'ROC-AUC Score with L2 regularization and lr={k}: {roc_auc}')

Convergence iteration with C=10: 46
ROC-AUC Score with L2 regularization and lr=0.01: 0.9362857142857142
Convergence iteration with C=10: 304
ROC-AUC Score with L2 regularization and lr=0.001: 0.9362857142857142
Convergence iteration with C=10: 1232
ROC-AUC Score with L2 regularization and lr=0.0001: 0.9366666666666666


Алгоритм не сходится если делать более длинные шаги lr > 0.1, и алгоритм градиентного спуска сходится медленее при более низком learning rate, но может попасть в другой локальный минимум, тем самым увеличивая качесвто метрики

#### 5. Попробуйте менять начальное приближение. Влияет ли оно на что-нибудь?

Изменим начальное приближение

In [7]:
predictions = log_regression(X=X, y=y, C=10, w1=-20.0, w2=10.0)

roc_auc = roc_auc_score(y, predictions)

print(f'ROC-AUC Score with L2 regularization and w1,w2_0 = [-20.0, 10.0]: {roc_auc}')

Convergence iteration with C=10: 7
ROC-AUC Score with L2 regularization and w1,w2_0 = [-20.0, 10.0]: 0.9362857142857142


Характер сходимости и сам факт сходимости метода градиентного спуска зависит от выбора начального приближения 